# Pipeline Test

Testing each step of my pipeline.

## Rough Outline (Jul. 7, 2025)
1. Grab playlist data from Spotify API.
   * The information available from the Spotify API should be enough. Features such as the number of releases, popularity, etc.
   * Later, in the UI itself, I would like the option to select a playlist instead of solely using a predefined one. This would then require the ability to automatically grab the playlist's most up-to-date features at any given moment. 
   * Note: I don't think I need features such as subgenres and audio features, as I believe those are likely already taken into account by lastfm in their recommendations to some extent. However, if needed, someone made an Exportify CLI (I have yet to try it) that may be useful here. It's just that as far as I'm concerned, I don't think it would do me any good to have those features if I cannot get those features for the similar artists. Also, Exportify yields song-level data.
2. Generate relevant features using Spotify API.
   * These (remember, artist-level) features would be things like: 
     * popularity (not sure how this is computed; I wonder if it would be better estimated via the total number of streams, although that would require more API requests and slow down the pipeline; lastfm has artist total playcounts that I can use instead)
     * number of releases
     * year of first release
     * year of last release
     * number of albums
     * number of tracks
     * number of times they show up in playlist (as primary artist or as a feature)
     * artist picture (for UI visualization purposes but obviously not a feature)
3. Generate relevant features using Lastfm API.
   * From `artist.getInfo` endpoint - things like: 
     * total listeners
     * total playcount
     * number of MY streams 
     * tags (genre tags; may or may not use; if I do use though, would need some way of filtering out irrelevant tags)
     * description/summary: for visualization purposes but also allows me to mine for nationality (+ record label, history, etc.) using some NLP technique if I'd like to use it as a feature or visualization tag
     * on tour: lastfm has a binary `ontour` label for each artist, although I think I want more detailed tour info, e.g., starting tour soon. 
4. Grab similar artists using Lastfm API.
   * In the final UI, I want to be able to toggle how many ripples outwards to go. Start with just one though; that would also be the ripple that contains the most immediate artists I should look into as they are deemed most similar to the ones I already like.
5. Grab touring data from external API - Songkick or Bandsintown.
6. Compute similarity/prioritization score using features and weights (GNN implementation).
   * Make sure that similar artists who are already in the playlist are not considered in the score. Or actually, would it make sense to weigh similar artists of those artists more heavily? That might make more sense actually.
   * Take into account the features listed in steps 2 and 3. Compute specific weights accordingly (proportion of artist songs in playlist, e.g.)
   * Generate rankings (top 5 or 10 highest similarity scorers)
7. Build UI. 
   * I want to have a means of visualizing the knowledge graph. I want to be able to select a node (an artist) and see their profile (basic info, their image, some of their general features)
   * I want the recommendations to have explainability - what features contributed to their being recommended?
   * Filtering options: 
      * Number of similar artist ripples outward
      * Minimum number of tracks an artist has in the playlist in order to be considered
      * Minimum number of my streams on that artist (^)
      * Which features to consider in the recommendations
    * I also want the ability to just directly select artists (2 or more) to generate the knowledge graph and recommendations for.
    * I would prefer not to use streamlit; I'm a bit tired of it and would prefer some novelty. I'm also not sure what frameworks even allow for visualization of knowledge graphs. I presume the KG can be produced in Plotly.
      * [Gradio](https://www.gradio.app/)
      * I want something similar to [this](https://huggingface.co/spaces/emilyalsentzer/SHEPHERD)
8. **Grab social media data; further prioritize artists commonly mentioned or recommended by social media accounts. This is a difficult thing to figure out and I will try it after I've finished everything else.

In [82]:
import pandas as pd
import numpy as np
import json
import spotipy
import re
import requests
import os
from dotenv import load_dotenv
from spotipy.oauth2 import SpotifyClientCredentials
load_dotenv()

True

### Step 1 - Grab playlist data from Spotify API.
* The information available from the Spotify API should be enough. Features such as the number of releases, popularity, etc.
* Later, in the UI itself, I would like the option to select a playlist instead of solely using a predefined one. This would then require the ability to automatically grab the playlist's most up-to-date features at any given moment.
* Note: I don't think I need features such as subgenres and audio features, as I believe those are likely already taken into account by lastfm in their recommendations to some extent. However, if needed, someone made an Exportify CLI (I have yet to try it) that may be useful here. It's just that as far as I'm concerned, I don't think it would do me any good to have those features if I cannot get those features for the similar artists. Also, Exportify yields song-level data.

--

Note that in the final UI, it would be nice to implement Spotify Oauth in order to fetch all of a user's (my) playlists. However it's not absoutely necessary; I could just provide the link to the playlist.

In [16]:
SPOTIFY_CLIENT_ID = os.environ.get("SPOTIFY_CLIENT_ID")
SPOTIFY_CLIENT_SECRET = os.environ.get("SPOTIFY_CLIENT_SECRET")

auth_manager = SpotifyClientCredentials(client_id=SPOTIFY_CLIENT_ID,
                                        client_secret=SPOTIFY_CLIENT_SECRET)
SPOTIFY = spotipy.Spotify(auth_manager=auth_manager)

In [26]:
def get_playlist_artists(playlist_url, client=SPOTIFY): 

    """Returns artist information associated with each track in the provided playlist.
       Note that each artist may appear multiple times."""    

    response = client.playlist_tracks(playlist_url, offset=0)
    # each artist involved in a song with features is considered individually
    artists = [artist_dict for track in response["items"] for artist_dict in track["track"]["artists"]]
    # total number of tracks on playlist - needed to determine the number of loops (max tracks per req. is 100)
    total = response["total"]

    for offset in range(100, total + 1, 100): 

        response = client.playlist_tracks(playlist_url, offset=offset)
        artists += [artist_dict for track in response["items"] for artist_dict in track["track"]["artists"]]

    return artists

In [25]:
artist_dicts = get_playlist_artists("https://open.spotify.com/playlist/142oZDOc1za2dkUwyonA1P?si=f651169be5044b4d", 
                                    SPOTIFY)
artist_dicts[:5]

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/3Ri4H12KFyu98LMjSoij5V'},
  'href': 'https://api.spotify.com/v1/artists/3Ri4H12KFyu98LMjSoij5V',
  'id': '3Ri4H12KFyu98LMjSoij5V',
  'name': 'Bad Omens',
  'type': 'artist',
  'uri': 'spotify:artist:3Ri4H12KFyu98LMjSoij5V'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/3Ri4H12KFyu98LMjSoij5V'},
  'href': 'https://api.spotify.com/v1/artists/3Ri4H12KFyu98LMjSoij5V',
  'id': '3Ri4H12KFyu98LMjSoij5V',
  'name': 'Bad Omens',
  'type': 'artist',
  'uri': 'spotify:artist:3Ri4H12KFyu98LMjSoij5V'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/3Ri4H12KFyu98LMjSoij5V'},
  'href': 'https://api.spotify.com/v1/artists/3Ri4H12KFyu98LMjSoij5V',
  'id': '3Ri4H12KFyu98LMjSoij5V',
  'name': 'Bad Omens',
  'type': 'artist',
  'uri': 'spotify:artist:3Ri4H12KFyu98LMjSoij5V'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/3Ri4H12KFyu98LMjSoij5V'},
  'href': 'https://api.spotify.com/v1/artis

### Step 2 - Generate relevant features using Spotify API.
* These (remember, artist-level) features would be things like: 
     * popularity (not sure how this is computed; I wonder if it would be better estimated via the total number of streams, although that would require more API requests and slow down the pipeline; lastfm has artist total playcounts that I can use instead)
     * number of releases
     * year of first release
     * year of last release
     * number of albums
     * number of tracks
     * number of times they show up in playlist (as primary artist or as a feature)
     * artist picture (for UI visualization purposes but obviously not a feature)

In [75]:
def get_artist_info(artist_dicts, client=SPOTIFY):

    """Returns Spotify API information associated with each unique artist, as 
       well as the artist count in the playlist. A consolidation, essentially."""

    artist_uris = set([artist["uri"] for artist in artist_dicts])
    all_artist_info = []

    for uri in artist_uris: 

        artist_info = client.artist(uri)
        artist_count = sum([1 for artist_dict in artist_dicts if artist_dict["uri"] == uri])
        artist_info["playlist_count"] = artist_count

        # album info
        albums = client.artist_albums(uri, include_groups="album")
        # number of albums
        artist_info["albums"] = albums["total"]
        album_items = albums["items"]
        # doesn't account for singles outside albums
        # don't want to count an album twice if a deluxe version was released
        # note that live versions are counted for now
        tracks_count = 0
        for album in album_items: 
             tracks_count += album["total_tracks"]
        artist_info["tracks"] = tracks_count
        
        artist_info["last_album_date"] = album_items[0]["release_date"] if album_items else None
        artist_info["first_album_date"] = album_items[-1]["release_date"] if album_items else None
    
        all_artist_info.append(artist_info)

    return all_artist_info

In [76]:
artist_info = get_artist_info(artist_dicts)
artist_info

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/4eMCK8Y88T54ZqWYnU51r5'},
  'followers': {'href': None, 'total': 104},
  'genres': [],
  'href': 'https://api.spotify.com/v1/artists/4eMCK8Y88T54ZqWYnU51r5',
  'id': '4eMCK8Y88T54ZqWYnU51r5',
  'images': [],
  'name': 'Bryan Kuznitz',
  'popularity': 27,
  'type': 'artist',
  'uri': 'spotify:artist:4eMCK8Y88T54ZqWYnU51r5',
  'playlist_count': 1,
  'albums': 0,
  'tracks': 0,
  'last_album_date': None,
  'first_album_date': None},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/4ZgQDCtRqZlhLswVS6MHN4'},
  'followers': {'href': None, 'total': 1206354},
  'genres': [],
  'href': 'https://api.spotify.com/v1/artists/4ZgQDCtRqZlhLswVS6MHN4',
  'id': '4ZgQDCtRqZlhLswVS6MHN4',
  'images': [{'url': 'https://i.scdn.co/image/ab6761610000e5eb1e5c7968999985162cad7042',
    'height': 640,
    'width': 640},
   {'url': 'https://i.scdn.co/image/ab676161000051741e5c7968999985162cad7042',
    'height': 320,
    'width': 320}

In [77]:
def get_spotify_features(artist_info, client=SPOTIFY):

    """Generates and returns features for each artist."""

    features = []

    for artist_dict in artist_info: 

        features.append({
            "name": artist_dict["name"],
            "uri": artist_dict["uri"],
            "genres": artist_dict["genres"],
            "albums": artist_dict["albums"],
            'tracks': artist_dict["tracks"],
            'last_album_date': artist_dict["last_album_date"],
            'first_album_date': artist_dict["first_album_date"],
            "popularity": artist_dict["popularity"],
            "followers": artist_dict["followers"]["total"],
            "playlist_count": artist_dict["playlist_count"],
            "spotify_url": artist_dict["external_urls"]["spotify"],
            "image_320": artist_dict["images"][1]["url"] if artist_dict["images"] else []
        })

    return features

In [78]:
spotify_features = get_spotify_features(artist_info)
spotify_features

[{'name': 'Bryan Kuznitz',
  'uri': 'spotify:artist:4eMCK8Y88T54ZqWYnU51r5',
  'genres': [],
  'albums': 0,
  'tracks': 0,
  'last_album_date': None,
  'first_album_date': None,
  'popularity': 27,
  'followers': 104,
  'playlist_count': 1,
  'spotify_url': 'https://open.spotify.com/artist/4eMCK8Y88T54ZqWYnU51r5',
  'image_320': []},
 {'name': 'grandson',
  'uri': 'spotify:artist:4ZgQDCtRqZlhLswVS6MHN4',
  'genres': [],
  'albums': 2,
  'tracks': 24,
  'last_album_date': '2023-05-05',
  'first_album_date': '2020-12-04',
  'popularity': 66,
  'followers': 1206354,
  'playlist_count': 6,
  'spotify_url': 'https://open.spotify.com/artist/4ZgQDCtRqZlhLswVS6MHN4',
  'image_320': 'https://i.scdn.co/image/ab676161000051741e5c7968999985162cad7042'},
 {'name': 'Noah Sebastian',
  'uri': 'spotify:artist:3w0i7biKO0q6DGVXCKzhj9',
  'genres': [],
  'albums': 0,
  'tracks': 0,
  'last_album_date': None,
  'first_album_date': None,
  'popularity': 33,
  'followers': 1314,
  'playlist_count': 1,
  'sp

In [80]:
spotify_features_df = pd.DataFrame(spotify_features)
spotify_features_df

,name,uri,genres,albums,tracks,last_album_date,first_album_date,popularity,followers,playlist_count,spotify_url,image_320
0,Bryan Kuznitz,spotify:artist:4eMCK8Y88T54ZqWYnU51r5,[],0,0,None,None,27,104,1,https://open.spotify.com/artist/4eMCK8Y88T54Zq...,[]
1,grandson,spotify:artist:4ZgQDCtRqZlhLswVS6MHN4,[],2,24,2023-05-05,2020-12-04,66,1206354,6,https://open.spotify.com/artist/4ZgQDCtRqZlhLs...,https://i.scdn.co/image/ab676161000051741e5c79...
2,Noah Sebastian,spotify:artist:3w0i7biKO0q6DGVXCKzhj9,[],0,0,None,None,33,1314,1,https://open.spotify.com/artist/3w0i7biKO0q6DG...,[]
3,Thornhill,spotify:artist:2dE5audbkV3o5cKxvM7lBD,"[metalcore, djent]",5,59,2025-04-04,2019-10-25,57,162050,10,https://open.spotify.com/artist/2dE5audbkV3o5c...,https://i.scdn.co/image/ab676161000051741b292d...
4,Lil West,spotify:artist:59dLTffLsw2ljmCiD8O1JX,[],3,25,2023-04-14,2019-03-15,41,24186,1,https://open.spotify.com/artist/59dLTffLsw2ljm...,https://i.scdn.co/image/ab676161000051746bebe7...
...,...,...,...,...,...,...,...,...,...,...,...,...
74,Fame on Fire,spotify:artist:10Z7WzKMeIdNBKexi1YarP,[],4,57,2024-09-06,2020-09-04,57,329660,1,https://open.spotify.com/artist/10Z7WzKMeIdNBK...,https://i.scdn.co/image/ab67616100005174f41b86...
75,Holding Absence,spotify:artist:2tl280wIokrLjabzrSKTgU,"[metalcore, post-hardcore, melodic hardcore]",4,51,2023-08-25,2019-03-01,57,171531,2,https://open.spotify.com/artist/2tl280wIokrLja...,https://i.scdn.co/image/ab67616100005174787bfd...
76,Amira Elfeky,spotify:artist:6Z4kwq0LOaYQW2CqudoOIk,[shoegaze],0,0,None,None,60,186217,2,https://open.spotify.com/artist/6Z4kwq0LOaYQW2...,https://i.scdn.co/image/ab67616100005174fa864e...
77,Caskets,spotify:artist:2XIbOWDT5vZPW7jNyzdfcK,"[metalcore, post-hardcore]",2,21,2023-08-11,2021-08-13,61,211293,6,https://open.spotify.com/artist/2XIbOWDT5vZPW7...,https://i.scdn.co/image/ab67616100005174d1f782...


In [81]:
spotify_features_df[spotify_features_df['name'] == "Bad Omens"]

,name,uri,genres,albums,tracks,last_album_date,first_album_date,popularity,followers,playlist_count,spotify_url,image_320
54,Bad Omens,spotify:artist:3Ri4H12KFyu98LMjSoij5V,[metalcore],6,83,2024-05-31,2016-08-19,75,1776694,48,https://open.spotify.com/artist/3Ri4H12KFyu98L...,https://i.scdn.co/image/ab676161000051743a62c7...


#### A couple notes for steps 1-2: 
* The specific genres are actually still intact - these are the same genres that Exportify yields. So I can actually still use these genres in my recommendations if I want.
* The album and track count for each artist do not take into account non-album singles and EPs. It gets a little messy if I try to count those, bc I can't really distinguish non-album vs album singles (efficiently).
* The album and track count will double count in the case of live or deluxe albums. Again, I don't see an efficient way to get rid of those. Of course I could filter out albums containing "deluxe" or "live" but that will remove albums who genuinely use those words in their title. I just don't think it's a big enough deal to address.
* <mark>The whole Spotify API pipeline is a little slow. Find some way to speed it up in actual implementation (consider parallelizing)</mark>

--
### Step 3 - Generate relevant features using Lastfm API.
* From `artist.getInfo` endpoint - things like: 
 * total listeners
 * total playcount
 * number of MY streams 
 * tags (genre tags; may or may not use; if I do use though, would need some way of filtering out irrelevant tags)
 * description/summary: for visualization purposes but also allows me to mine for nationality (+ record label, history, etc.) using some NLP technique if I'd like to use it as a feature or visualization tag
 * on tour: lastfm has a binary `ontour` label for each artist, although I think I want more detailed tour info, e.g., starting tour soon. 

In [83]:
LASTFM_API_KEY = os.environ.get("LASTFM_API_KEY")

In [98]:
artist = "bad+omens"
url = f"https://ws.audioscrobbler.com/2.0/?method=artist.getinfo&artist={artist}&username={username}&api_key={LASTFM_API_KEY}&format=json"
response = requests.get(url)
omens = response.json()

In [104]:
artist = "dquwhdihsiuadiwjfoiwq"
username = "jasminexx18"
url = f"https://ws.audioscrobbler.com/2.0/?method=artist.getinfo&artist={artist}&username={username}&api_key={LASTFM_API_KEY}&format=json"
response = requests.get(url)
response.json()

{'error': 6,
 'message': 'The artist you supplied could not be found',
 'links': []}

In [108]:
# if nothing is found, just ignore (too much work to get MBIDs and try again, esp. given
# that Lastfm's MBIDs are sparsely populated
def get_lastfm_features(artist_names, username="jasminexx18", api_key=LASTFM_API_KEY): 

    """Grabs Lastfm features for a list of artists. Goes by artist name."""

    artist_info = []
    
    for artist_name in artist_names: 

        url = f"https://ws.audioscrobbler.com/2.0/?method=artist.getinfo&artist={artist_name}&username={username}&api_key={api_key}&format=json"

        try: 
            response = requests.get(url).json()["artist"]
            
            artist_info.append({
                "name": artist_name,
                "lastfm_listeners": response["stats"]["listeners"],
                "lastfm_playcount": response["stats"]["playcount"],
                "personal_playcount": response["stats"]["userplaycount"],
                "lastfm_tags": [tag["name"] for tag in response["tags"]["tag"]] if response["tags"]["tag"] else [],
                "summary": response["bio"]["summary"]
            })

        except Exception as e: 
            print(f"Artist {artist_name} not found on lastfm.")

    return artist_info

In [107]:
lastfm_features = get_lastfm_features(spotify_features_df["name"])
lastfm_features

[{'name': 'Bryan Kuznitz',
  'lastfm_listeners': '3',
  'lastfm_playcount': '6',
  'personal_playcount': '0',
  'lastfm_tags': [],
  'summary': ' <a href="https://www.last.fm/music/Bryan+Kuznitz">Read more on Last.fm</a>'},
 {'name': 'grandson',
  'lastfm_listeners': '715458',
  'lastfm_playcount': '20897336',
  'personal_playcount': '62',
  'lastfm_tags': ['indie rock',
   'alternative rock',
   'alternative',
   'seen live',
   'trap'],
  'summary': 'Grandson is an alternative artist hailing from Toronto, Canada. Born in the small town of Englewood New Jersey, he relocated to the cultural melting pot of Toronto at a young age, and grew up surrounded by music ranging from jazz to rock and roll to rap, dancehall and r&b. He began playing guitar and piano in high school.\n\nAt 17, he relocated to Montreal, QC to attend university, and began working in nightclubs cleaning tables and DJing. He started writing music at this time, incorporating the unique blend of sounds he grew up surround

In [109]:
lastfm_features_df = pd.DataFrame(lastfm_features)
lastfm_features_df

,name,lastfm_listeners,lastfm_playcount,personal_playcount,lastfm_tags,summary
0,Bryan Kuznitz,3,6,0,[],"<a href=""https://www.last.fm/music/Bryan+Kuzn..."
1,grandson,715458,20897336,62,"[indie rock, alternative rock, alternative, se...",Grandson is an alternative artist hailing from...
2,Noah Sebastian,28,217,0,[],"Noah Sebastian Davis (born October 31, 1995 in..."
3,Thornhill,160864,10548682,425,"[metalcore, alternative metal, Progressive Met...",After the band – comprising of Jacob Charlton ...
4,Lil West,41233,402891,0,"[Hip-Hop, rap, emo, underground hip-hop, trap]",22 year old Delaware rapper Lil West is an art...
...,...,...,...,...,...,...
74,Fame on Fire,175893,3957971,1,"[post-hardcore, alternative rock, metalcore, r...",Formed in 2013 in the theme park capital of th...
75,Holding Absence,214847,8165498,13,"[post-hardcore, seen live, alternative rock, e...","Holding Absence is a rock band from Cardiff, W..."
76,Amira Elfeky,205251,3774758,22,"[Nu Metal, rock, alternative metal, female voc...",Amira Lynne Elfeky is an American-Egyptian sin...
77,Caskets,234222,6909495,164,"[post-hardcore, metalcore, alternative rock, s...","Caskets, formed in 2018 under the name Captive..."


In [110]:
all_features = spotify_features_df.merge(lastfm_features_df, how="left", on="name")
all_features

,name,uri,genres,albums,tracks,last_album_date,first_album_date,popularity,followers,playlist_count,spotify_url,image_320,lastfm_listeners,lastfm_playcount,personal_playcount,lastfm_tags,summary
0,Bryan Kuznitz,spotify:artist:4eMCK8Y88T54ZqWYnU51r5,[],0,0,None,None,27,104,1,https://open.spotify.com/artist/4eMCK8Y88T54Zq...,[],3,6,0,[],"<a href=""https://www.last.fm/music/Bryan+Kuzn..."
1,grandson,spotify:artist:4ZgQDCtRqZlhLswVS6MHN4,[],2,24,2023-05-05,2020-12-04,66,1206354,6,https://open.spotify.com/artist/4ZgQDCtRqZlhLs...,https://i.scdn.co/image/ab676161000051741e5c79...,715458,20897336,62,"[indie rock, alternative rock, alternative, se...",Grandson is an alternative artist hailing from...
2,Noah Sebastian,spotify:artist:3w0i7biKO0q6DGVXCKzhj9,[],0,0,None,None,33,1314,1,https://open.spotify.com/artist/3w0i7biKO0q6DG...,[],28,217,0,[],"Noah Sebastian Davis (born October 31, 1995 in..."
3,Thornhill,spotify:artist:2dE5audbkV3o5cKxvM7lBD,"[metalcore, djent]",5,59,2025-04-04,2019-10-25,57,162050,10,https://open.spotify.com/artist/2dE5audbkV3o5c...,https://i.scdn.co/image/ab676161000051741b292d...,160864,10548682,425,"[metalcore, alternative metal, Progressive Met...",After the band – comprising of Jacob Charlton ...
4,Lil West,spotify:artist:59dLTffLsw2ljmCiD8O1JX,[],3,25,2023-04-14,2019-03-15,41,24186,1,https://open.spotify.com/artist/59dLTffLsw2ljm...,https://i.scdn.co/image/ab676161000051746bebe7...,41233,402891,0,"[Hip-Hop, rap, emo, underground hip-hop, trap]",22 year old Delaware rapper Lil West is an art...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,Fame on Fire,spotify:artist:10Z7WzKMeIdNBKexi1YarP,[],4,57,2024-09-06,2020-09-04,57,329660,1,https://open.spotify.com/artist/10Z7WzKMeIdNBK...,https://i.scdn.co/image/ab67616100005174f41b86...,175893,3957971,1,"[post-hardcore, alternative rock, metalcore, r...",Formed in 2013 in the theme park capital of th...
75,Holding Absence,spotify:artist:2tl280wIokrLjabzrSKTgU,"[metalcore, post-hardcore, melodic hardcore]",4,51,2023-08-25,2019-03-01,57,171531,2,https://open.spotify.com/artist/2tl280wIokrLja...,https://i.scdn.co/image/ab67616100005174787bfd...,214847,8165498,13,"[post-hardcore, seen live, alternative rock, e...","Holding Absence is a rock band from Cardiff, W..."
76,Amira Elfeky,spotify:artist:6Z4kwq0LOaYQW2CqudoOIk,[shoegaze],0,0,None,None,60,186217,2,https://open.spotify.com/artist/6Z4kwq0LOaYQW2...,https://i.scdn.co/image/ab67616100005174fa864e...,205251,3774758,22,"[Nu Metal, rock, alternative metal, female voc...",Amira Lynne Elfeky is an American-Egyptian sin...
77,Caskets,spotify:artist:2XIbOWDT5vZPW7jNyzdfcK,"[metalcore, post-hardcore]",2,21,2023-08-11,2021-08-13,61,211293,6,https://open.spotify.com/artist/2XIbOWDT5vZPW7...,https://i.scdn.co/image/ab67616100005174d1f782...,234222,6909495,164,"[post-hardcore, metalcore, alternative rock, s...","Caskets, formed in 2018 under the name Captive..."


In [111]:
all_features.to_csv("hardnheavy_features_prelim.csv", index=False)